In [3]:
import pandas as pd
from pandas.io import gbq
import numpy as np
import mysql.connector
import pandas as pd
import numpy as np
from pyhive import presto
import datetime
from dateutil.relativedelta import relativedelta
from TCLIService.ttypes import TOperationState
import sqlalchemy
import oyoms as om

conn = presto.connect(host='presto.oyorooms.io',port=8889,username='vikram.singh6@oyorooms.com',schema = 'ingestiondb')


Building Connection with workbook on Onedrive

In [4]:
excel_link = 'https://oyoenterprise-my.sharepoint.com/:x:/g/personal/finance_analyst_oyorooms_com/ER6hwRJkBohKpXtAROZ9uzsBhiw3hk_QjYQRZcpMLfLR7w?e=nQDMCh'
wb = om.WorkbookClient('vikram.singh6@oyorooms.com', excel_link)

In [7]:
Qry_1 = """

select distinct 
cast((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute as date) Data_Date,
hs.hub_name,
date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H') Hours,
sum(date_diff('day',date(checkin),date(checkout))) urns,           
sum(coalesce(bb.amount ,bb.amount_f,0) - coalesce(bb.discount ,bb.discount_f,0)) RT
from ordering_realtime.bookings bb
inner join aggregatedb.hotels_summary hs on bb.hotel_id = hs.hotel_id
where bb.inserted_at >= date_format(date_trunc('month', current_date) - interval '1' day,'%Y%m') 
and cast(cast(bb.created_at as timestamp) as date) >= date_trunc('month', ((now()  + interval '5' hour) + interval '30' minute)) - interval '1' day
and hs.country_id = 1
group by 1,2,3
order by date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H')


"""

Qry_2 = """

select distinct 
(date_trunc('month', ((now() + interval '5' hour) + interval '30' minute)) - interval '1' day) Data_Date,
hs.hub_name,
date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H') Hours,
sum(date_diff('day',date(checkin),date(checkout))) urns,           
sum(coalesce(bb.amount ,bb.amount_f,0) - coalesce(bb.discount ,bb.discount_f,0)) RT
from ordering_realtime.bookings bb
inner join aggregatedb.hotels_summary hs on bb.hotel_id = hs.hotel_id
where bb.inserted_at >= date_format(date_trunc('month', current_date) - interval '1' day,'%Y%m') 
and cast(cast(bb.created_at as timestamp) as date) between date_trunc('month',(date_trunc('month', ((now()  + interval '5' hour) + interval '30' minute)) - interval '1' day)) and (date_trunc('month', ((now()  + interval '5' hour) + interval '30' minute)) - interval '1' day)
and hs.country_id = 1
group by 1,2,3
order by date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H')


"""

Qry_3 = """

select distinct 
(((now()  + interval '5' hour) + interval '30' minute) - interval '7' day) Data_Date,
hs.hub_name,
date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H') Hours,
sum(date_diff('day',date(checkin),date(checkout))) urns,           
sum(coalesce(bb.amount ,bb.amount_f,0) - coalesce(bb.discount ,bb.discount_f,0)) RT
from ordering_realtime.bookings bb
inner join aggregatedb.hotels_summary hs on bb.hotel_id = hs.hotel_id
where bb.inserted_at >= date_format(date_trunc('month', current_date) - interval '1' day,'%Y%m') 
and (cast(cast(bb.created_at as timestamp) as date) between (((now()  + interval '5' hour) + interval '30' minute) - interval '14' day) and  (((now()  + interval '5' hour) + interval '30' minute) - interval '7' day))
and hs.country_id = 1
group by 1,2,3
order by date_format((cast(bb.created_at as timestamp) + interval '5' hour) + interval '30' minute,'%H')

"""

Qry_4 = """

select 
max(((cast(created_at as timestamp) + interval '5' hour) + interval '30' minute))
from 
ordering_realtime.bookings

"""




In [8]:
df_1 = pd.read_sql_query(Qry_1,conn)
df_2 = pd.read_sql_query(Qry_2,conn)
df_3 = pd.read_sql_query(Qry_3,conn)
df_1=df_1.append(df_2)
df_1=df_1.append(df_3)
wb.clear_range('RAW',range_address='A:E',apply_to='ALL')
wb.write_data(sheet_name = 'RAW', df = df_1, location = 'A1', include_header = True)
df_4 = pd.read_sql_query(Qry_4,conn)
wb.write_value('Live ARR Dashboard', range_address='Z1', value=pd.to_datetime(datetime.datetime.today()))

In [ ]:
del df_1,df_2,df_3,df_4